# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

#define API key
import configparser
config = configparser.ConfigParser()
initfile = "..\\config\\auth.ini"
config.read(initfile)
g_key = config.get('GoogleAPIKey','g_key')

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
cities_df = pd.read_csv("../output_data/cities.csv", encoding='utf-8').drop(['Unnamed: 0'],axis=1)
cities_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,vaini,TO,-21.2000,-175.2000,80.60,83,75,5.75,1619065677
1,kapaa,US,22.0752,-159.3190,73.99,94,90,8.05,1619065694
2,provideniya,RU,64.3833,-173.3000,35.60,55,0,3.56,1619065695
3,egvekinot,RU,66.3167,-179.1667,32.20,78,100,1.74,1619065686
4,alofi,NU,-19.0595,-169.9187,84.20,74,96,8.05,1619065893
...,...,...,...,...,...,...,...,...,...
539,ngunguru,NZ,-35.6167,174.5000,66.00,81,15,1.99,1619066314
540,russell,NZ,32.3502,-85.1999,48.20,49,1,5.75,1619066314
541,asau,TV,46.4333,26.4000,48.99,100,0,2.30,1619065987
542,beringovskiy,RU,63.0500,179.3167,29.17,79,67,4.23,1619065961


In [14]:
len((cities_df[cities_df["Latitude"]<-90])|(cities_df[cities_df["Latitude"]>90]))

0

In [19]:
locations = cities_df[['Latitude','Longitude']]
weights = cities_df['Humidity (%)'].astype(float)
max_humidity = cities_df['Humidity (%)'].max()
print(weights)
locations 

0       83.0
1       94.0
2       55.0
3       78.0
4       74.0
       ...  
539     81.0
540     49.0
541    100.0
542     79.0
543     75.0
Name: Humidity (%), Length: 544, dtype: float64


,Latitude,Longitude
0,-21.2000,-175.2000
1,22.0752,-159.3190
2,64.3833,-173.3000
3,66.3167,-179.1667
4,-19.0595,-169.9187
...,...,...
539,-35.6167,174.5000
540,32.3502,-85.1999
541,46.4333,26.4000
542,63.0500,179.3167


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width':'900px',
    'height':'500px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout = figure_layout)
heat_layer = gmaps.heatmap_layer(locations,weights= weights,dissipating=False,max_intensity=max_humidity.astype(float),point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
#Find the cities with ideal weather
ideal_weather_cities_df = cities_df.loc[(cities_df["Max Temperature (F)"]>70)& (cities_df["Max Temperature (F)"]<80)&(cities_df["Wind Speed (mph)"]<10)&(cities_df["Cloudiness (%)"]==0)].dropna()
ideal_weather_cities_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
167,serra,BR,-20.1286,-40.3078,71.60,88,0,3.44,1619066262
178,vila velha,BR,-20.3297,-40.2925,71.60,88,0,3.44,1619065698
180,maceio,BR,-9.6658,-35.7353,75.20,94,0,3.44,1619066263
205,atar,MR,20.5169,-13.0499,73.24,22,0,9.28,1619066212
285,sohag,EG,26.5570,31.6948,73.40,31,0,6.91,1619066279
309,sakakah,SA,29.9697,40.2064,78.80,18,0,9.22,1619066283


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# create hotel_df
hotel_df = ideal_weather_cities_df[["City","Country","Latitude","Longitude"]]
#Add a "Hotel Name" column
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-36-ac843bfe75cb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Latitude,Longitude,Hotel Name
167,serra,BR,-20.1286,-40.3078,
178,vila velha,BR,-20.3297,-40.2925,
180,maceio,BR,-9.6658,-35.7353,
205,atar,MR,20.5169,-13.0499,
285,sohag,EG,26.5570,31.6948,
309,sakakah,SA,29.9697,40.2064,


In [37]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "radius" : 5000,  #search for hotels within 5000 meters of city lat & lng
          "key" : g_key}

for index,row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    for i in range(0,len(response['results'])):
        try:
            if response['results'][i]["business_status"] == "OPERATIONAL":
                hotel_name = response['results'][i]['name']
                hotel_df.loc[index,"Hotel Name"] = hotel_name
                print(f"The nearest operational hotel to city {city.title()} is hotel {hotel_name}")
                break
            else:
                continue
        except KeyError:
            continue
    
hotel_df

c:\users\helena\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


The nearest operational hotel to city Serra is hotel Emef Serrana
The nearest operational hotel to city Vila Velha is hotel Quality Suites Vila Velha
The nearest operational hotel to city Maceio is hotel Clínicas de Saúde Cesmac
The nearest operational hotel to city Atar is hotel Oumou Elghoura
The nearest operational hotel to city Sohag is hotel Doctors Club
The nearest operational hotel to city Sakakah is hotel Fakhamat Al Orjoana Apartment


,City,Country,Latitude,Longitude,Hotel Name
167,serra,BR,-20.1286,-40.3078,Emef Serrana
178,vila velha,BR,-20.3297,-40.2925,Quality Suites Vila Velha
180,maceio,BR,-9.6658,-35.7353,Clínicas de Saúde Cesmac
205,atar,MR,20.5169,-13.0499,Oumou Elghoura
285,sohag,EG,26.5570,31.6948,Doctors Club
309,sakakah,SA,29.9697,40.2064,Fakhamat Al Orjoana Apartment


In [43]:
# rename Latitude and Longitude columns to Lat and Lng
hotel_df = hotel_df.rename(columns ={"Latitude":"Lat","Longitude":"Lng"})
hotel_df

,City,Country,Lat,Lng,Hotel Name
167,serra,BR,-20.1286,-40.3078,Emef Serrana
178,vila velha,BR,-20.3297,-40.2925,Quality Suites Vila Velha
180,maceio,BR,-9.6658,-35.7353,Clínicas de Saúde Cesmac
205,atar,MR,20.5169,-13.0499,Oumou Elghoura
285,sohag,EG,26.5570,31.6948,Doctors Club
309,sakakah,SA,29.9697,40.2064,Fakhamat Al Orjoana Apartment


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…